In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/Rinbow

/content/drive/MyDrive/Rinbow


In [3]:
#import dependencies library

# the enviornment library
import gym

#the AI framework 
import tensorflow as tf
import tensorflow.keras as keras 
import numpy as np

#just for ploting stuf
import cv2
import matplotlib.pyplot as plt
import tqdm

from collections import deque
from utils import *

In [4]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars

Streaming output truncated to the last 5000 lines.
Extracting  ROMS/Barnstorming (1982) (Activision, Steve Cartwright) (AX-013) ~.bin       3%  OK 
Extracting  ROMS/Barnstorming (CCE).bin                                    3%  OK 
Extracting  ROMS/Barnstorming (Unknown) (PAL) [a].bin                      3%  OK 
Extracting  ROMS/Barnstorming (Unknown) (PAL).bin                          3%  OK 
Extracting  ROMS/Barnstorming - Die tollkeuhnen Flieger (1982) (Activision, Steve Cartwright - Ariola) (EAX-013, PAX-013 - 711 013-720) (PAL).bin       3%  OK 
Extracting  ROMS/Base Attack (1983) (Home Vision - Gem International Corporation - VDI) (VCS83113) (PAL) ~.bin       3%  OK 
Extracting  ROMS/Base Attack (Hack) (Unknown).bin                          3%  OK 
Extracting  ROMS/Baseball (AKA Super Challenge Baseball) (1988) (Telegames) (5665 A016) (PAL).bin       3%  OK 
Extracting  ROMS/Baseball (AKA Super Challenge Base

In [5]:
class FireResetEnv(gym.Wrapper):
    """
    This class is Warpper where we make sure that the game is runing, becouse in some environment in gym library should 
    start the game by action " 1 " in most of the game so by doing this we make sure that the game is roling.
    """
    def __init__(self, env=None):
        super(FireResetEnv, self).__init__(env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3
        
    def step(self, action):
        return self.env.step(action)
    
    def reset(self):
        self.env.reset()
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset()
            obs, _, done, _ = self.env.step(2)
            if done:
                self.env.reset()
        return obs

In [6]:
environment=gym.make("Breakout-v4")
env=FireResetEnv(environment)

In [7]:
keras.backend.clear_session()

class CategoricalDQN(keras.Model):
    def __init__(self,atoms,actions,**kwargs):
        super().__init__(**kwargs)
        
        #The conv layers in the model 
        self.conv=keras.Sequential([
            keras.layers.Conv2D(32,8,strides=4,padding='valid',activation='relu'),
            keras.layers.Conv2D(64,4,strides=2,padding='valid',activation='relu'),
            keras.layers.Conv2D(64,3,strides=1,padding='valid',activation='relu'),
            keras.layers.Flatten()
        ])
        self.V_h1=FactorisedNoisyLayer(512,activation='relu')
        self.A_h1=FactorisedNoisyLayer(512,activation='relu')
        self.state_value=FactorisedNoisyLayer(atoms)
        self.raw_advantage=FactorisedNoisyLayer(atoms*actions)
        self.atoms=atoms
        self.actions=actions
        
    def call(self,x):
        #find the batch this important if we try to pass one example or more so the batch is not fixed.
        batch_size=x.shape[0]
        
        conv_out = self.conv(x)
        
        value_hidd = self.V_h1(conv_out)
        adva_hidd = self.A_h1(conv_out)
        
        value_out = self.state_value(value_hidd)
        r_advantage = self.raw_advantage(adva_hidd)
        
        reshape_advantage = tf.reshape(r_advantage,[batch_size,-1,self.atoms])
        
        advantage = reshape_advantage - tf.reduce_mean(reshape_advantage,axis=2,keepdims=True)
        
        # the output is [batch_size,Action*AToms] if actions is 4 the the shape is [batch_size,204]
        value_out_= value_out[:,None ,:]
        
        out = value_out_ + advantage
        
        # the reshape step were we reshape the output 
#         out = tf.reshape(out,[batch_size,-1,ATOMS])
        # apply the softmax function to the distribuation 
        return keras.activations.softmax(out,axis=2)

In [9]:
model=CategoricalDQN(51,4)
model_2=CategoricalDQN(51,4)

In [10]:
model_2.build([32,84,84,4])
model.build([32,84,84,4])

In [11]:
class Agent:
    def __init__(self,environment,model,model2,discount_factor=0.99,learning_rate=0.00025,episode=10000,batch_size=32,steps=3,replay_size=1000000,staked_frame=4,fixed_epsilon=None,model_checkpoint='DQN_wights.h5',up=25,down=10,left=5,right=5,update_steps=50):
        #the environment simulator 
        self.env=environment
        # hear we made the first change where we have 2 model instead of one model the online_model and target_model 
        #  the first one it's using for compute Q_value function at each time step while the other to just compute the target.
        self.online_model=model
        # the second model is the target model have the same structure of the online model and the same inial parameter.
        self.target_model=model2
        # the discout factor in the Q-learning algorithem
        self.discount_factor=discount_factor
        # the number of episod to run as the number of epochs in supervised learning 
        self.episodes=episode
        # onather update to the previouse alogorithem where we need parameter that difine the update rate for the target model.
        self.update_rate=update_steps
    
        self.batch_size=batch_size
        # the memory where we save the observations for training 
        self.replay_buffer=Memory(replay_size)
        # the deque used for state creation where we stacked last 4 frames togather 
        self.stacked_blocks=deque(maxlen=staked_frame)
        # if we want to stack more or less frame togather in other world if we want the state more complex or simpler.
        self.stacked_size=staked_frame
        # if we want the epsilon parameter constant during the training else the parameter will change during the training.
        self.fixed_epsilon=fixed_epsilon
        # the action space it's represent the number neuron in the output layer becouse Q function is maping from state to actions. 
        self.action_space=environment.action_space.n
        # it's rewards list where we save the reward from each episod ( from each epochs )
        self.rewards_list=[]
        # the loss function using to compute the gradiant of the model is KLD not MSE and we compute manualy.
        # self.loss_function=tf.losses.mean_squared_error
#---->   the optimizer using to update the parameter is no longer RMSprop.
        self.optimizer=tf.optimizers.Adam(learning_rate=learning_rate,epsilon=1.5e-4)
        # it's the file path, where we save the model weights
        self.model_checkpoint=model_checkpoint
        #number of steps to play 
        self.steps=steps
        # these four parameter we use for croping the edges of the frame  
        self.up=up
        self.down=-down
        self.left=left
        self.right=-right
    
    def _preprocess_frame(self,frame):
        """
        this function make preprocessing step for each frame from the game.
        parameter: 
            -frame: is a row frame with [210*64*3] size which provided by the environment, after we make an action that change the world
            the environmnt brovide us by new observation after apply that action.
        return :
            image: processing frame where we cut some of border and keep the play area and change the image to gray scale 
            then resize it to [84,84] shape and finaly normalize pixel value to become in the range [0-1] 
        """
        # cut the porder and keep the play area
        image=frame[self.up:self.down,self.left:self.right,:]
        image=tf.image.rgb_to_grayscale(image)
        image=tf.image.resize(image,[84,84])
        # reshape becouse it's tensor [84,84,1] and we want [84,84] for make the stacking operation easy
        image=tf.reshape(image,[84,84])
        # normalizing step
        image=image/255.0
        return image
    
    def ploting_function_one(frame):
        """
        is the same as preprocessing function but we use it for viualization stuf.
        """
        image=frame[25:,:,:]
        image=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
        image=cv2.resize(image,(85,85))
        return image
    
    
    def _state_creator(self,frame,is_new_episod):
        """
        this function we stack last 4 frame togather to perform on state and that give us some intuition about vilocity.
        parameter :
            -frame :which is the game observation.
            -is_new_episod: boolean parameter that check if there is no previous frame and that happend in bragning of each episod.
        
        return :
            state: tensor with [84,84,4] shape which represent the state of our world.
        """
        # first preprocessing the frame.
        image=self._preprocess_frame(frame)
        # if it's new episod then stack the first frame four time 
        if is_new_episod:
            for i in range(self.stacked_size):
                self.stacked_blocks.append(image)
                
        # just push the last frame so the first frame get out from the deque
        else:
            self.stacked_blocks.append(image)
            
        # stacked the frames togather in one tensor [84,84,4]
        state=tf.stack(self.stacked_blocks,axis=2)
        
        return state
    
    def _sample_experiences(self):
        """
        This function is use to sample batch from our memory.
        
        return:
            -states:it's tensore [batch_size,image_width,image_height,stacked_size] in our case [32,84,84,4]
            -actions: it's 2D array [batch_size,action] it's the action in each state in our batch
            -rewards: it's 2D array [batch_size,reward] it's the reward in each state in out batch
            -dones : it's 2D array [batch_size, done ] where done is boolean value help us to compute the target.
            -next_state: it's tensore [batch_size,image_width,image_height,stacked_size] in our case [32,84,84,4]
        """
#---->  instead of using random choice to select transition we use the prioritized memory object to sample data.
        batch,weights,indices=self.replay_buffer.sample_experiences(self.batch_size)
        # we combine the experiance togather where the buffer have tuple like this (state,action,reward,done,next_state)
        states,actions,rewords,dones,next_states=[np.array([experiance[field_index] for experiance in batch]) for field_index in range(5)]
#---->  we return the weights and inices becouse we need them for training step. 
        return states,actions,rewords,dones,next_states,weights,indices
    
    
    
    def noisy_policy(self,state):
        """
        Is the epislon greedy policy whre we use epsilon value to chose an action
        where we want to palance the exploration and explotation as possiable as we can.

        we pick random value alpha.
            -if alpha < epsilon : chose random action
             else argmax Q(state) for all action.
        parameter :
            -state: is the current state.
            -epsilon : is the value of threshould between [0,1]
        return :
            the number of action to make 

        """
        dist=self.online_model(state[np.newaxis])
        Q_values=tf.reduce_sum(dist*Z,axis=2) 
        # a* = argmax Q(xt+1,a) over a   the shape is [Batch , 1]
        #best_next_actions=tf.argmax(next_Q_values,axis=1).numpy()
        return tf.argmax(Q_values[0])
    
    def _play_multi_steps(self,state):
        """
        This function make an action and get the observation from the world.
        we make an action and then get the full obesrvation from our environment aplay state_create function to the our 
        observation(frame) and then save it in the memory for training step.
        
        parameter :
            -env: it's represent our world or environment we work on 
            -state : it's the current state.
            -epsilon: it's the thresould we use for action selection
        return :
            next_state: tensore [84,84,4] block represent the last 4 frame.
            reward : it's float number the reward we get after we do some action . 
            done : it's boolean refer if the game finish or not 
            info : it's dictionary have the counter of lives.
        """
        rewards_n=0
        next_state=np.copy(state)
        action=None
        for i in range(self.steps):
            action_n=self.noisy_policy(next_state)
            if not action:
                action=action_n
            next_state,reward,done,info=env.step(action_n)
            if info['ale.lives'] < 5:
                done=True
                break
            next_state=self._state_creator(next_state,False)
            rewards_n += self.discount_factor**i + reward 
        if done:    
            next_state=self._state_creator(next_state,False)
        self.replay_buffer.add((state,action,rewards_n,done,next_state))
        return next_state,reward,done,info
        
    def _training_step(self):
        """
        This function is the responsible for doing one training step where we sample one batch from the replay buffer 
        then we push this batch in our model for forward step and apply gradiant with optimizer by hand.
        
        return :
            None , where it's apply the change inplace for model parameter.
        """
        
        # get the transition (xt,at,rt,donet,xt+1) , gamma_t in [0,1] 
#---->  get the batch from the replay_buffer (our memory) with weights and indices.
        states,actions,rewards,dones,next_states,weights,indices=self._sample_experiences()
        # find the propability of the next state the output shape is [batch,action,atoms] and hear we use model in the agent we will use target model 
        # we need the batch size
        batch_size=states.shape[0]
        # hear we calculate p(xt+1,a) but for the whole BATCH 
        next_value_dist=self.online_model(next_states) 
        # this is the Q(Xt+1,a) = Sum_i (zi*pi(xt+1,a)) the shape is [Batch,Actions] this is the Q_value for all actions 
        next_Q_values=tf.reduce_sum(next_value_dist*Z,axis=2) 
        # a* = argmax Q(xt+1,a) over a   the shape is [Batch , 1]
        best_next_actions=tf.argmax(next_Q_values,axis=1).numpy()
        # find the action-value distribuation for just the best action pj(xt+1,a*)
        next_value_dist = self.target_model(next_states)
        
        next_value_dist=tf.stack([ next_value_dist[index,best_next_actions[index]] for index in np.arange(batch_size)]) # batch,ATOMS 
        #
        target_distribuation = distribution_projection(next_value_dist,rewards,self.discount_factor,dones)

        with tf.GradientTape() as tape:
            # compute the Q function for current state (hear for the whole batch).
            all_value_dist=self.online_model(states)
            # as befor find the value distribuation for the action we made 
            action_value_dist=tf.stack([ all_value_dist[index,actions[index]] for index in np.arange(batch_size)])
            #computer the log for the propability distribuation for finding the loss 
            # log Pi(xt,at)
            log_action_value=tf.math.log(action_value_dist)
            # compute the loss using KLD 
            losses= tf.reduce_sum(-log_action_value*target_distribuation,axis=1)
            
            loss = tf.reduce_mean(losses * weights)
        # compute the gradient for our  online_model parameter.
        gradiants=tape.gradient(loss,self.online_model.trainable_variables)
        # apply the optimization step using the gradient we compute for the online_model
        self.optimizer.apply_gradients(zip(gradiants,self.online_model.trainable_variables))
        
        self.replay_buffer.update(indices,losses)

        
    def fit(self):
        """
        This function represent the training loop function where we are trying to fit model to produce the best function
        according to the data we have.
        
        """
        for episode in tqdm.tqdm(range(self.episodes)):
            state=self.env.reset()
            stacked_state=self._state_creator(state,True)
            rewards=0
            epsilon=max(1-episode/7000,0.01)
            while True:
                stacked_state,reward,done,info=self._play_multi_steps(stacked_state)
                rewards+=reward
                if done :
                    if not (episode%10) and len(self.rewards_list):
                        # self.model.save_weights(self.model_checkpoint)
                        print("The best score for last 10 episode is: {} and the worst one is: {}".format(max(self.rewards_list[-10:]),min(self.rewards_list[-10:])))
                    self.rewards_list.append(rewards)
                    break
                # this step to let the replay buffer has some experiance
                if episode>50:
                    self._training_step()
                #hear where update the target model after 50 episod by set the parameter as online model.
                if episode%self.update_rate:
                    self.target_model.set_weights(self.online_model.get_weights())

In [12]:
agent=Agent(env,model,model_2)

In [ ]:
agent.fit()

  0%|          | 11/10000 [00:17<3:34:40,  1.29s/it]

The best score for last 10 episode is: 0.0 and the worst one is: 0.0


  0%|          | 21/10000 [00:26<2:41:20,  1.03it/s]

The best score for last 10 episode is: 0.0 and the worst one is: 0.0


  0%|          | 31/10000 [00:37<2:46:25,  1.00s/it]

...

  0%|          | 41/10000 [00:47<2:44:36,  1.01it/s]

The best score for last 10 episode is: 0.0 and the worst one is: 0.0


  1%|          | 51/10000 [00:57<2:34:20,  1.07it/s]

The best score for last 10 episode is: 0.0 and the worst one is: 0.0


  1%|          | 52/10000 [01:10<12:41:21,  4.59s/it]